In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Detect torch & CUDA versions
import torch
TORCH = torch.__version__.split('+')[0]            # e.g. "2.1.0"
CUDA  = 'cu' + torch.version.cuda.replace('.', '') # e.g. "cu118"
print("Installing for torch", TORCH, "and", CUDA)

# Install the pre-built extensions
!pip install --quiet torch-scatter   -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install --quiet torch-sparse    -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install --quiet torch-cluster   -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html
!pip install --quiet torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

# Finally install PyG itself
!pip install --quiet torch-geometric

Installing for torch 2.6.0 and cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### If done on colab, make sure to cd to the location/repo that contain all the files. 

### Below are example usages of short scripts for quick valiation and a long script for more comprehensive ablation studies.

In [6]:
# Standard GCN baseline
!python amp_gnn_experiment.py --model gcn --dataset Cora --hid 16 --epochs 200 --lr 0.01

Epoch 050 | Loss 0.0739 | Train 1.0000 | Val 0.7640 | Test 0.7880 | 1.5s elapsed
Epoch 100 | Loss 0.0381 | Train 1.0000 | Val 0.7780 | Test 0.8010 | 1.8s elapsed
Epoch 150 | Loss 0.0357 | Train 1.0000 | Val 0.7700 | Test 0.8060 | 2.0s elapsed
Epoch 200 | Loss 0.0218 | Train 1.0000 | Val 0.7700 | Test 0.8040 | 2.3s elapsed

Final Test Accuracy: 0.8070
Total time: 2.3s


In [7]:
!python amp_gnn_experiment.py \
  --model amp \
  --agg add \
  --damping 0.2 \
  --use_skip \
  --hid 16 \
  --epochs 200 \
  --lr 0.01 \
  --wd 5e-4 \
  --seed 0

Epoch 050 | Loss 25.3676 | Train 0.8571 | Val 0.5340 | Test 0.5400 | 0.8s elapsed
Epoch 100 | Loss 81.6840 | Train 0.9357 | Val 0.6540 | Test 0.6730 | 1.2s elapsed
Epoch 150 | Loss 3247.6470 | Train 0.7286 | Val 0.3960 | Test 0.4120 | 1.6s elapsed
Epoch 200 | Loss 3015.0168 | Train 0.7929 | Val 0.5220 | Test 0.5290 | 2.0s elapsed

Final Test Accuracy: 0.7280
Total time: 2.0s


In [9]:
!python amp_gnn_experiment.py \
  --model amp \
  --lamp \
  --agg add \
  --damping 0.2 \
  --use_skip \
  --hid 16 \
  --epochs 200 \
  --lr 0.01 \
  --wd 5e-4 \
  --seed 0

Epoch 050 | Loss 252.8925 | Train 0.6786 | Val 0.3520 | Test 0.3070 | 0.8s elapsed
Epoch 100 | Loss 141.3209 | Train 0.9143 | Val 0.5760 | Test 0.5850 | 1.2s elapsed
Epoch 150 | Loss 112.5754 | Train 0.9786 | Val 0.6140 | Test 0.6360 | 1.6s elapsed
Epoch 200 | Loss 689.8181 | Train 0.9429 | Val 0.6500 | Test 0.6850 | 2.0s elapsed

Final Test Accuracy: 0.6850
Total time: 2.0s


In [11]:
!python amp_gnn_experiment.py \
  --model amp --agg add --damping 0.2 \
  --hid 16 --epochs 200 --lr 0.01 --wd 5e-4 --seed 0

Epoch 050 | Loss 50.8019 | Train 0.4357 | Val 0.3860 | Test 0.3480 | 0.8s elapsed
Epoch 100 | Loss 1022.2671 | Train 0.1429 | Val 0.1620 | Test 0.1490 | 1.1s elapsed
Epoch 150 | Loss 1519.7878 | Train 0.1429 | Val 0.1620 | Test 0.1490 | 1.5s elapsed
Epoch 200 | Loss 1.9459 | Train 0.1429 | Val 0.1220 | Test 0.1300 | 1.9s elapsed

Final Test Accuracy: 0.4450
Total time: 1.9s


In [12]:
# LAMP variant with residual skips and mixed aggregator
!python amp_gnn_experiment.py --model amp --lamp --use_skip --agg mix --damping 0.5 --memory 2 --hid 16 --epochs 200

Epoch 050 | Loss 24418265088.0000 | Train 0.5071 | Val 0.1540 | Test 0.1690 | 0.9s elapsed
Epoch 100 | Loss 4102855540253210443776.0000 | Train 0.5143 | Val 0.2880 | Test 0.3060 | 1.3s elapsed
Epoch 150 | Loss 265757118413975784436051723943936.0000 | Train 0.5429 | Val 0.2880 | Test 0.2970 | 1.8s elapsed
Epoch 200 | Loss nan | Train 0.1429 | Val 0.1220 | Test 0.1300 | 2.3s elapsed

Final Test Accuracy: 0.2940
Total time: 2.3s


In [ ]:
%%bash
cat > run_ablation.sh << 'EOF'
#!/usr/bin/env bash
# Cora ablation: GCN vs AMP variants
DATASET="Cora"
EPOCHS=200; LR=0.01; WD=5e-4
SEEDS=(0 1 2)
DAMPINGS=(0.2 0.5 0.8)

echo "# Model Seed Damping Skip LAMP Memory Accuracy" > results.txt

# Baseline GCN
for seed in "${SEEDS[@]}"; do
  ACC=$(python amp_gnn_experiment.py \
    --model gcn --dataset $DATASET --hid 16 --epochs $EPOCHS \
    --lr $LR --wd $WD --seed $seed \
    | grep "Final Test Accuracy" | awk '{print $4}')
  echo "GCN $seed - No No 0 $ACC" >> results.txt
done

# AMPConv variants
for damping in "${DAMPINGS[@]}"; do
  for seed in "${SEEDS[@]}"; do
    # plain AMPConv
    ACC=$(python amp_gnn_experiment.py --model amp --dataset $DATASET --hid 16 \
      --epochs $EPOCHS --lr $LR --wd $WD --damping $damping --memory 0 --seed $seed \
      | grep "Final Test Accuracy" | awk '{print $4}')
    echo "AMPConv $seed $damping No No 0 $ACC" >> results.txt

    # + skip
    ACC=$(python amp_gnn_experiment.py --model amp --dataset $DATASET --hid 16 \
      --epochs $EPOCHS --lr $LR --wd $WD --damping $damping --memory 0 \
      --use_skip --seed $seed \
      | grep "Final Test Accuracy" | awk '{print $4}')
    echo "AMPConv $seed $damping Yes No 0 $ACC" >> results.txt

    # + LAMP
    ACC=$(python amp_gnn_experiment.py --model amp --dataset $DATASET --hid 16 \
      --epochs $EPOCHS --lr $LR --wd $WD --damping $damping --memory 0 \
      --use_skip --lamp --seed $seed \
      | grep "Final Test Accuracy" | awk '{print $4}')
    echo "AMPConv $seed $damping Yes Yes 0 $ACC" >> results.txt

    # + memory
    ACC=$(python amp_gnn_experiment.py --model amp --dataset $DATASET --hid 16 \
      --epochs $EPOCHS --lr $LR --wd $WD --damping $damping --memory 1 \
      --use_skip --seed $seed \
      | grep "Final Test Accuracy" | awk '{print $4}')
    echo "AMPConv $seed $damping Yes No 1 $ACC" >> results.txt
  done
done

# VAMPBlock
for seed in "${SEEDS[@]}"; do
  ACC=$(python amp_gnn_experiment.py --model vamp --dataset $DATASET --hid 16 \
    --epochs $EPOCHS --lr $LR --wd $WD --seed $seed \
    | grep "Final Test Accuracy" | awk '{print $4}')
  echo "VAMPBlock $seed - No No 0 $ACC" >> results.txt
done

echo "Done!"
EOF

chmod +x run_ablation.sh
./run_ablation.sh


Done!
